In [2]:
from functools import partial
from typing import Callable

import autorootcwd  # noqa: F401

import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

from scipy.stats import loguniform
# from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import (
    RandomizedSearchCV,
    RepeatedStratifiedKFold,
    train_test_split,
)
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder

from xgboost import XGBClassifier

# Predictive task definition

The objective from these experiments is to the identify the machine's patterns through Machine Learing models. A way to interpret this objective, is to visualize its sensor data, generate interesting features, and use those features at a machine learning model to predict its classes.

Two possible problems approaches: 
- attribute a single class for each sample (multi-class classfication problem); 
- consider that multiple faults can happen at the same time (multi-label classification, can be considered binary classification for each fault).

The latter problem is more realistic considering a few factors: 
- a multi-class approach fails when new classes appear (there must be an "others" class);
- multiple faults DO happen in practice;

But there are a few problems:
- For this dataset, the classes are labeled in a way that each sample only has a single fault happening. Therefore, it is not fair to the multi-label model;
- We turn a perfectly stratified dataset into a imbalanced dataset. Different metrics such as balanced accuracy, or ROC AUC must be used;
- More parameters to tune, such as the threshold for the ROC curve. It is possible to fix a certain metric such as TPR to diminish FNs, trading-off for FPR.

A few considerations:
- it would be better to have different machines/sensors and have them separated in the training/testing sets to avoid a possibe data leakage, where the model learns specific configurations of the machine. It is not possible to do this with this dataset. 
- the classes are perfectly stratified for a multi-class approach, but will turn imbalanced when turning the problem into a multi-label.

## Experiment propositions:

Initially, the problem is going to be defined as a multi-class classification and the following experiments are going to be made:

1. Train a simple baseline linear model (LogisticRegression) on only a subset of features (time-domain statistical features);
2. Retrain the same baseline model on other subsets of features (time-domain impulsive, spectral, and envelope features);
3. Train the baseline model with all features;

Results: comparison table from experiments 1, 2, and 3.

4. Explore different more capable models (RF, SVM, XGBoost, CatBoost, etc.);

Results: comparison table with all the different model results.

After those experiments, repeat for the multi-label approach.

## [Exp. 1] Baseline model

In [3]:
feature_dataset = pd.read_parquet('data/processed/specialized_features.parquet')

#### [Exp. 1.1]

In [4]:
vibration_sensor_cols = feature_dataset.columns[feature_dataset.columns.str.contains('sensor_1|sensor_2|sensor_3')]
vibration_features = feature_dataset[vibration_sensor_cols]

In [5]:
statistical_feat_cols = vibration_features.columns[vibration_features.columns.str.contains('rms|peak_to_peak|kurtosis|skewness|crest_factor')]
statistical_features = vibration_features[statistical_feat_cols]

impulsive_feat_cols = vibration_features.columns[vibration_features.columns.str.contains('peak|impulse_factor|crest_factor|clearance_factor')]
impulsive_features = vibration_features[impulsive_feat_cols]

only_time_domain = vibration_features.columns[vibration_features.columns.str.contains('acceleration|velocity')]
time_domain_statistical_features = vibration_features[only_time_domain]

only_frequency_domain = vibration_features.columns[vibration_features.columns.str.contains('fft|envelope')]
frequency_domain_features = vibration_features[only_frequency_domain]

In [6]:
only_frequency_domain

Index(['sensor_1/fft/rms/5-500', 'sensor_2/fft/rms/5-500',
       'sensor_3/fft/rms/5-500', 'sensor_1/fft/rms/500-1000',
       'sensor_2/fft/rms/500-1000', 'sensor_3/fft/rms/500-1000',
       'sensor_1/fft/rms/5-1000', 'sensor_2/fft/rms/5-1000',
       'sensor_3/fft/rms/5-1000', 'sensor_1/fft/rms/500-1500',
       'sensor_2/fft/rms/500-1500', 'sensor_3/fft/rms/500-1500',
       'sensor_1/fft/rms/1000-1500', 'sensor_2/fft/rms/1000-1500',
       'sensor_3/fft/rms/1000-1500', 'sensor_1/fft/rms/1500-2000',
       'sensor_2/fft/rms/1500-2000', 'sensor_3/fft/rms/1500-2000',
       'sensor_1/fft/rms/2000-3000', 'sensor_2/fft/rms/2000-3000',
       'sensor_3/fft/rms/2000-3000', 'sensor_1/fft/rms/3000-5000',
       'sensor_2/fft/rms/3000-5000', 'sensor_3/fft/rms/3000-5000',
       'sensor_1/fft/peak/5-500', 'sensor_2/fft/peak/5-500',
       'sensor_3/fft/peak/5-500', 'sensor_1/fft/peak/500-1000',
       'sensor_2/fft/peak/500-1000', 'sensor_3/fft/peak/500-1000',
       'sensor_1/fft/peak/5-100

In [7]:
time_domain_statistical_features

,sensor_1/acceleration/mean,sensor_2/acceleration/mean,sensor_3/acceleration/mean,sensor_1/acceleration/std,sensor_2/acceleration/std,sensor_3/acceleration/std,sensor_1/acceleration/max,sensor_2/acceleration/max,sensor_3/acceleration/max,sensor_1/acceleration/min,...,sensor_3/velocity/peak,sensor_1/velocity/impulse_factor,sensor_2/velocity/impulse_factor,sensor_3/velocity/impulse_factor,sensor_1/velocity/crest_factor,sensor_2/velocity/crest_factor,sensor_3/velocity/crest_factor,sensor_1/velocity/clearance_factor,sensor_2/velocity/clearance_factor,sensor_3/velocity/clearance_factor
0,0.009764,0.011494,0.006821,0.095766,0.161612,0.283912,0.338075,0.511311,0.674950,-0.223562,...,0.000207,2.433362,3.808749,3.469636,2.037123,2.974232,2.751640,2.791162,4.570390,4.195728
1,0.009800,0.002345,0.006794,0.098875,0.212293,0.275367,0.198619,0.612137,0.749696,-0.306607,...,0.000138,2.204137,3.182531,2.569735,1.929710,2.579658,2.093326,2.456032,3.740394,3.035847
2,0.005349,0.011791,0.005558,0.010677,0.020596,0.023571,0.027556,0.056871,0.056572,-0.020569,...,0.000026,1.936146,2.458270,2.257664,1.683775,2.093925,1.920331,2.143697,2.742221,2.561180
3,0.000187,-0.005966,-0.001712,0.085607,0.196855,0.205195,0.235776,0.569074,0.463487,-0.209703,...,0.000143,2.343853,2.638394,3.109306,2.007188,2.181265,2.583273,2.636515,3.039421,3.574973
4,0.006093,0.002964,0.001560,0.060175,0.175737,0.192483,0.170310,0.660217,0.606119,-0.185975,...,0.000118,2.273364,4.294731,3.318239,1.954634,3.354862,2.553922,2.557923,5.052758,4.074405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.018096,0.004166,0.015179,0.093106,0.196701,0.245160,0.251402,0.627928,0.907333,-0.191248,...,0.000150,2.740211,3.570448,3.403280,2.311937,2.860436,2.700949,3.145440,4.209266,4.045308
49996,0.008015,0.010936,0.014510,0.070580,0.151584,0.219644,0.167889,0.456616,0.498169,-0.179111,...,0.000140,2.479343,3.028907,3.034161,2.193811,2.449043,2.435532,2.709995,3.506853,3.585953
49997,0.004533,0.011163,-0.000664,0.092091,0.199724,0.259247,0.217659,0.588285,0.710894,-0.214861,...,0.000172,2.932770,3.177637,2.931674,2.442035,2.507414,2.393400,3.372916,3.843365,3.432871
49998,0.016410,0.019973,0.037972,0.097714,0.194672,0.262104,0.235953,0.568399,0.829214,-0.248728,...,0.000131,2.528983,3.559742,2.786614,2.103129,2.874283,2.343532,2.923512,4.182241,3.157713


### Label encoding

In [7]:
dataset = time_domain_statistical_features

In [8]:
enc = LabelEncoder()
labels = enc.fit_transform(feature_dataset.classes)

features = dataset.values

In [9]:
labels.shape, labels

((50000,), array([3, 0, 0, ..., 3, 0, 1]))

In [10]:
features.shape

(50000, 84)

### Stratified train-test split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

In [12]:
pd.Series(y_test).value_counts()

4    2000
2    2000
3    2000
0    2000
1    2000
Name: count, dtype: int64

### Hparam Optimization

In [71]:
random_state = 42

param_dist_lr = {
    "param__C": loguniform(1e-3, 1e3),
}


repeated_stratified_kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=random_state)
lr = LogisticRegression(max_iter=2000, random_state=random_state)

model_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('param', lr)
])

search_strategy = RandomizedSearchCV(
    estimator=model_pipeline,
    param_distributions=param_dist_lr,
    scoring="accuracy",
    cv=repeated_stratified_kfold,
    n_iter=20,
    n_jobs=-1,
    return_train_score=True,
    random_state=random_state,
)

random_search_cv_results = search_strategy.fit(X_train, y_train)

In [75]:
score_optimization = {
    "hparam_opt/train_cv/acc": random_search_cv_results.cv_results_["mean_train_score"][random_search_cv_results.best_index_],
    "hparam_opt/test_cv/acc": random_search_cv_results.cv_results_["mean_test_score"][random_search_cv_results.best_index_],
}

In [76]:
random_search_cv_results.best_params_

{'param__C': 506.1576888752306}

In [77]:
random_search_cv_results.score(X_test, y_test)

0.4941

In [8]:
class Trainer:
    def __init__(self, datamodule, model):
        self.model = model
        self.X_train, self.X_test, self.y_train, self.y_test = datamodule.split()

    def fit_cv(self, search_strategy, param_distributions):
        # self.logger.info(f"Training {self.model.__name__}...")
        if self.model.__name__ == "LogisticRegression":
            model = Pipeline(
                steps=[("scaler", StandardScaler()), ("model", self.model(max_iter=2000, random_state=42))]
            )
            param_distributions = {
                f"param__{k}": v for k, v in param_distributions.items()
            }
        else:
            model = self.model
        search_strategy = search_strategy(
            estimator=model, param_distributions=param_distributions
        )

        # self.logger.info("Optimizing hyperparameters...")
        search_strategy.fit(self.X_train, self.y_train)

        # self.logger.info("Retrieving hparams and results...")
        hparams = search_strategy.best_params_
        opt_score = {
            "hparam_opt/train_cv/acc": search_strategy.cv_results_["mean_train_score"][
                search_strategy.best_index_
            ],
            "hparam_opt/test_cv/acc": search_strategy.cv_results_["mean_test_score"][
                search_strategy.best_index_
            ],
        }
        
        test_score = {"test/acc": search_strategy.score(self.X_test, self.y_test)}
        
        return opt_score, test_score, hparams

    def fit(self, X_train, y_train, hparams):
        # self.logger.info(f"Training {self.model.__name__}...")
        if self.model.__name__ == "LogisticRegression":
            hparams = {k.split("__")[1]: v for k, v in hparams.items()}

            model = Pipeline(
                steps=[("scaler", StandardScaler()), ("model", self.model(**hparams))]
            )
        else:
            model = self.model(**hparams)

        model.fit(X_train, y_train)
        return model

    @staticmethod
    def predict(model, X_test):
        y_test = model.predict(X_test)
        return y_test

    @staticmethod
    def evaluate(y_test, y_pred):
        score = {
            "test/acc": accuracy_score(y_test, y_pred),
        }
        return score

In [9]:
class DataModule:
    def __init__(self, dataset, label_col, feature_cols, test_size, random_state):
        self.dataset = dataset
        self.label_col = label_col
        self.feature_cols = feature_cols
        self.test_size = test_size
        self.random_state = random_state
        
    def prepare_data(self):
        features = self.dataset[self.feature_cols].values
        labels = self.dataset[self.label_col].values
        return features, labels
    
    def encode_labels(self, labels, problem_type="multi-class"):
        if problem_type == "multi-class":
            enc = LabelEncoder()
            labels = enc.fit_transform(labels)
        elif problem_type == "multi-label":
            raise NotImplementedError(" Multi-label classification is not yet supported")
        else:
            raise ValueError("Invalid problem type. Please choose either 'multi-class' or 'multi-label'")
        return labels
    
    def split(self):
        features, labels = self.prepare_data()
        labels = self.encode_labels(labels)
        X_train, X_test, y_train, y_test = train_test_split(
            features, labels, test_size=self.test_size, random_state=self.random_state, stratify=labels
        )
        return X_train, X_test, y_train, y_test

In [10]:
def search_strategy(
    estimator: Callable,
    param_distributions: dict,
    search_algo: Callable,
    cv_method: Callable,
    n_iter: int,
    random_state: int,
    n_jobs: int,
    return_train_score: bool,
):
    strategy = search_algo(
        estimator=model_pipeline,
        param_distributions=param_distributions,
        scoring="accuracy",
        cv=cv_method,
        n_iter=n_iter,
        n_jobs=-1,
        return_train_score=return_train_score        
    )
    return strategy

In [233]:
lr = LogisticRegression(max_iter=2000, random_state=random_state)

repeated_stratified_kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=random_state)

param_dist_lr = {
    "C": loguniform(1e-3, 1e3),
}

partial_search_strategy = partial(
    search_strategy,
    search_algo=RandomizedSearchCV,
    cv_method=repeated_stratified_kfold,
    n_iter=20,
    random_state=random_state,
    n_jobs=-1,
    return_train_score=True,
)

In [234]:
datamodule = DataModule(
    feature_dataset,
    label_col="classes",
    feature_cols=only_time_domain,
    test_size=0.2,
    random_state=42,
)

In [235]:
model = LogisticRegression

trainer = Trainer(datamodule, LogisticRegression)

In [237]:
opt_scores, test_score, hparams = trainer.fit_cv(search_strategy=partial_search_strategy, param_distributions=param_dist_lr)

In [238]:
opt_scores

{'hparam_opt/train_cv/acc': 0.49053541666666667,
 'hparam_opt/test_cv/acc': 0.4855083333333334}

In [240]:
test_score

{'test/acc': 0.4938}

In [241]:
hparams

{'param__C': 10.289871888521818}

In [216]:
_,X_test, _, y_test = datamodule.split()

In [226]:
test_model = trainer.fit(model, X_test, y_test, hparams)

/home/rodrigo-kobashikawa/miniconda3/envs/study-case/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [228]:
y_pred = trainer.predict(test_model, X_test)

In [229]:
trainer.evaluate(y_test, y_pred)

{'test/acc': 0.489}

### XGBoost

In [41]:
datamodule = DataModule(
    feature_dataset,
    label_col="classes",
    feature_cols=feature_dataset.drop(columns=['classes']).columns,
    test_size=0.2,
    random_state=42,
)

In [42]:
trainer = Trainer(model=XGBClassifier, datamodule=datamodule)

In [43]:
X_train, X_test, y_train, y_test = datamodule.split()

In [44]:
model = trainer.fit(X_train, y_train, hparams={"n_jobs": -1, "random_state": 42})

In [45]:
model.score(X_train, y_train)

0.997725

In [46]:
model.score(X_test, y_test)

0.8926

### Catboost

In [11]:
from catboost import CatBoostClassifier

In [12]:
datamodule = DataModule(
    feature_dataset,
    label_col="classes",
    feature_cols=feature_dataset.drop(columns=['classes']).columns,
    test_size=0.2,
    random_state=42,
)

In [13]:
trainer = Trainer(model=CatBoostClassifier, datamodule=datamodule)

In [14]:
X_train, X_test, y_train, y_test = datamodule.split()

In [15]:
model = trainer.fit(X_train, y_train, hparams={"random_seed": 42})

Learning rate set to 0.095505
0:	learn: 1.5171074	total: 121ms	remaining: 2m
1:	learn: 1.4410924	total: 169ms	remaining: 1m 24s
2:	learn: 1.3795957	total: 213ms	remaining: 1m 10s
3:	learn: 1.3262198	total: 257ms	remaining: 1m 3s
4:	learn: 1.2825018	total: 300ms	remaining: 59.8s
5:	learn: 1.2415880	total: 344ms	remaining: 57.1s
6:	learn: 1.2028528	total: 387ms	remaining: 54.9s
7:	learn: 1.1729158	total: 431ms	remaining: 53.4s
8:	learn: 1.1424844	total: 475ms	remaining: 52.3s
9:	learn: 1.1164374	total: 518ms	remaining: 51.3s
10:	learn: 1.0921005	total: 566ms	remaining: 50.8s
11:	learn: 1.0698532	total: 608ms	remaining: 50.1s
12:	learn: 1.0487618	total: 652ms	remaining: 49.5s
13:	learn: 1.0293057	total: 696ms	remaining: 49s
14:	learn: 1.0109378	total: 740ms	remaining: 48.6s
15:	learn: 0.9940189	total: 783ms	remaining: 48.1s
16:	learn: 0.9780457	total: 826ms	remaining: 47.8s
17:	learn: 0.9631117	total: 869ms	remaining: 47.4s
18:	learn: 0.9500518	total: 911ms	remaining: 47s
19:	learn: 0.937

In [16]:
model.score(X_train, y_train)

0.96815

In [17]:
model.score(X_test, y_test)

0.8963